# Pygsti report
(Created on August 03, 2017)

In [0]:
import numpy as np
import pygsti
from pygsti.construction import std1Q_XYI

#The usual GST setup: we're going to run GST on the standard XYI 1-qubit gateset
gs_target = std1Q_XYI.gs_target
fiducials = std1Q_XYI.fiducials
germs = std1Q_XYI.germs
maxLengths = [1,2,4,8]
listOfExperiments = pygsti.construction.make_lsgst_experiment_list(
    gs_target.gates.keys(), fiducials, fiducials, germs, maxLengths)

#Create some datasets for analysis
gs_datagen1 = gs_target.depolarize(gate_noise=0.1, spam_noise=0.001)
gs_datagen2 = gs_target.depolarize(gate_noise=0.05, spam_noise=0.01).rotate(rotate=0.01)

ds1 = pygsti.construction.generate_fake_data(gs_datagen1, listOfExperiments, nSamples=1000,
                                            sampleError="binomial", seed=1234)
ds2 = pygsti.construction.generate_fake_data(gs_datagen2, listOfExperiments, nSamples=1000,
                                            sampleError="binomial", seed=1234)
ds3 = ds1.copy_nonstatic(); ds3.add_counts_from_dataset(ds2); ds3.done_adding_data()

#Run GST on all three datasets
gs_target.set_all_parameterizations("TP")
results1 = pygsti.do_long_sequence_gst(ds1, gs_target, fiducials, fiducials, germs, maxLengths, verbosity=0)
results2 = pygsti.do_long_sequence_gst(ds2, gs_target, fiducials, fiducials, germs, maxLengths, verbosity=0)
results3 = pygsti.do_long_sequence_gst(ds3, gs_target, fiducials, fiducials, germs, maxLengths, verbosity=0)


In [0]:
results   = results1
estimates = results.estimates
default   = estimates['default']
gatesets  = default.gatesets

tgt      = gatesets['target']
ds       = results.dataset
gs       = gatesets['go0']
gs_final = gatesets['final iteration estimate']
#gs_final = results1.gatestring_structs['final']

cri = None


In [0]:
from pygsti.report import Workspace
ws = Workspace()
ws.init_notebook_mode(connected=False, autodisplay=True)



### Model Violation summary: 
The aggregate log-likelihood as a function of GST iterations''')


In [0]:

#ws.FitComparisonBarPlot(Ls, gs_finalAllL, switchBd.gsAllL, 
#                        eff_ds, switchBd.objective, 'L')



### Per-sequence model violation: 
Each point displays the goodness of fit for a single gate sequence.''')


In [0]:

'''
colorScatterPlot = ws.ColorBoxPlot(
    switchBd.objective, gs_final, eff_ds, gsL,
    linlg_pcntle=float(linlogPercentile) / 100,
    minProbClipForWeighting=switchBd.mpc, scatter=True) #TODO: L-switchboard on summary page?
colorScatterPlot.set_render_options(click_to_display=True)
'''



### Comparison of GST estimated gates to target gates: 
This table presents, for each of the gates, three different measures of distance or discrepancy from the GST estimate to the ideal target operation.  See text for more detail.''')


In [0]:

ws.GatesVsTargetTable(gs_final, gs_target, cri)



# Model Violation Analysis
Metrics indicating how well the estimated gate set can be trusted -- i.e., how well it fits the data.
### ProgressTable:
Comparison between the computed and expected maximum $\log(\mathcal{L})$ for different values of $L$. 
$N_S$ and $N_p$ are the number of gate strings and parameters, respectively.  
The quantity $2\Delta\log(\mathcal{L})$ measures the goodness of fit of the GST model (small is better) and is expected to lie within $[k-\sqrt{2k},k+\sqrt{2k}]$ where $k = N_s-N_p$. 
$N_\sigma = (2\Delta\log(\mathcal{L})-k)/\sqrt{2k}$ is the number of standard deviations from the mean (a $p$-value can be straightforwardly derived from $N_\sigma$).  
The rating from 1 to 5 stars gives a very crude indication of goodness of fit.

  <!--<span class="math">p</span> is the p-value derived from a <span class="math">\chi^2_k</span> distribution.(For example, if <span class="math">p=0.05</span>, then the probability of observing a <span class="math">\chi^{2}</span> value as large as, or larger than, the one indicated in the table is 5%%, assuming the GST model is valid.) -->



In [0]:

ws.FitComparisonBarPlot(Ls, gs_finalAllL, switchBd.gsAllL, 
                        eff_ds, switchBd.objective, 'L')



### bestEstimateColorBoxPlot:


In [0]:

'''
colorScatterPlot = ws.ColorBoxPlot(
    switchBd.objective, gs_final, eff_ds, gsL,
    linlg_pcntle=float(linlogPercentile) / 100,
    minProbClipForWeighting=switchBd.mpc, scatter=True) #TODO: L-switchboard on summary page?
colorScatterPlot.set_render_options(click_to_display=True)
'''
#switchBd.view(maxLView,"v6")




### $2\Delta\log(\mathcal{L})$ contributions for every individual experiment in the dataset.
Each pixel represents a single experiment (gate sequence), and its color indicates whether GST was able to fit the corresponding frequency well.  Shades of white/gray are typical. Red squares represent statistically significant evidence for model violation (non-Markovianity), and should appear with probability at most *linlg_pcntle* if the data really are Markovian. Square blocks of pixels correspond to base sequences (arranged vertically by germ and horizontally by length); each pixel within a block corresponds to a specific choice of pre- and post-fiducial sequences.  See text for further details.




### dataScalingColorBoxPlot


In [0]:

ws.ColorBoxPlot("scaling", switchBd.gs_finalFinal, eff_ds, switchBd.gs_finalIter,
        submatrices=switchBd.scaledSubMxsDict)



### Data scaling factor for every individual experiment in the dataset.
Each pixel represents a single experiment (gate sequence), and its color indicates the amount of scaling that was applied to the original data counts when computing the log-likelihood or $\chi^2$ for this estimate.  
Values of 1.0 indicate all of the original data was used, whereas numbers between 0 and 1 indicate that the data counts for the experiement were artificially decreased (usually to improve the fit).


 
# Gauge Invariant Outputs
Quantities which are *gauge-invariant* are the most reliable means of assessing the quality of the gates, as these do not depend on any unphysical gauge degrees of freedom




### Eigenvalues of estimated gates: 
The spectrum (Eigenvalues column) of each estimated gate.  The second column displays these eigenvalues over the complex disc.


In [0]:

ws.GateEigenvalueTable(gs_final, gs_target, cri, display=('evals','log-evals'))




### Gram Matrix Eigenvales: Compares the eigenvalues of the data-derived Gram matrix with those of a Gram matrix computed using the target gates.


In [0]:
 
ws.GramMatrixBarPlot(ds1, gs_target, 10)#, strs)



### Gauge Variant Outputs
The raw estimated gate set, and then some useful derived quantities.  These quanties are <q>gauge-dependent</q>, meaning they will depend on unphysical gauge degrees of freedom that are a necessary byproduct of estimating an entire gate set at once (akin to a freedom of reference frame).  After finding a best-estimate based on the (physical) data, GST optimizes within the space of all (unphysical) gauge degrees of freedom using the parameters in Table



In [0]:

# Choose G.O. params
# goSwitchBoard2




### Gauge Optimization Details. 
A list of the parameters used when performing the gauge optimization that produced the final GST results found in subsequent tables and figures.



In [0]:

ws.GaugeOptParamsTable(switchBd.goparams)




### The GST estimate of the SPAM operations. Compares the estimated SPAM operations to those desired for convenience.



In [0]:

ws.SpamTable(switchBd.gsTargetAndFinal,
             ['Target','Estimated'],
             cri, includeHSVec=False)




### GST estimate of SPAM probabilities.
Computed by taking the dot products of vectors in bestGatesetSpamTable. The symbol $E_C$, when it appears, refers to the <q>complement</q> effect given by subtracting each of the other effects from the identity.



In [0]:

ws.SpamParametersTable(gs_final, cri)




### Decomposition of estimated gates.
A rotation axis and angle are extracted from each gate by considering the projection of its logarithm onto a the Pauli Hamiltonian projectors.  The direction and magnitude (up to a conventional constant) give the axis and angle respectively.


In [0]:

ws.GateDecompTable(gs_final, gs_target, cri)




### Relative Eigenvalues of estimated gates.
The spectrum of each estimated gate pre-multiplied by the inverse of it's ideal (target) counterpart.  The second column displays these eigenvalues over the complex disc.



In [0]:

ws.GateEigenvalueTable(gs_final, gs_target, cri, display=('rel','log-rel'))





### Comparison of GST estimated gates to target gates:
This table presents, for each of the gates, three different measures of distance or discrepancy from the GST estimate to the ideal target operation.  See text for more detail.



In [0]:

ws.GatesVsTargetTable(gs_final, gs_target, cri)




### Comparison of GST estimated SPAM to target SPAM.
This table presents, for each state preparation and POVM effect, two different measures of distance or discrepancy from the GST estimate to the ideal target operation.  See text for more detail.



In [0]:

ws.SpamVsTargetTable(gs_final, gs_target, cri)




### The GST estimate of the logic gate operations.
Compares the *ideal* (generally unitary) logic gates (second column, also in targetGatesetTable) with those *estimated* by GST (third column).  Each gate is represented as a $d^2\times d^2$ *superoperator* that acts by matrix multiplication on vectors in $\mathcal{B}(\mathcal{H})$.  Matrices are displayed using a heat map that ranges between 1.0 (red) and -1.0 (blue).  Note that it is impossible to discern even order-1%% deviations from the ideal using this color scale, and one should rely on other analysis for more a precise comparison.



In [0]:

ws.GatesTable(switchBd.gsTargetAndFinal,
              ['Target','Estimated'], "boxes", cri)



# Dataset comparisons
This report contains information for more than one data set.  This page shows comparisons between different data sets.




### Histogram of p-values comparing two data sets. 
Each gate sequence is assigned a p-value based on how consistent that sequence's counts are between the two selected data sets.  The line shows what would be expected for perfectly consistent data.



In [0]:

ws.DatasetComparisonPlot(dscmp_switchBd.dscmp)




# Per-sequence $2\Delta\log(\mathcal{L})$ values comparing two data sets.
In a similar fashion to other color box plots, this plot shows two times the log-likelihood-ratio for each gate sequence corresponding to how consistent that sequences' counts are between the two selected data sets.  The likelihood ratio is between a models that supposes there is either one or two separate probability distributions from which the data counts are drawn.



In [0]:

ws.ColorBoxPlot('dscmp', dscmp_switchBd.dscmp_gs_final,
                None, None, dscomparator=dscmp_switchBd.dscmp)



# Input Summary
Information pertaining to the target gate set and data set.



### Target gate set: SPAM (state preparation and measurement).
The *ideal* input state ($\rho_0$) and `plus' POVM effect $E_0$ for the device on which we report.  SPAM gates are given here as $d\times d$ matrices.



In [0]:

ws.SpamTable(gs_target, None, includeHSVec=False)



### Fiducial sequences.
A list of the preparation and measurement <q>fiducial</q> gate sequences. See discussion in text.


In [0]:
 
ws.GatestringTable(strs,["Prep.","Measure"], commonTitle="Fiducials")




### Germ sequences.
A list of the <q>germ</q> gate sequences.  See discussion in text.



In [0]:

ws.GatestringTable(germs, "Germ", nCols=2)




### General dataset properties.
See discussion in text.



In [0]:

ws.DataSetOverviewTable(ds)




### Target gate set: logic gates.
The *ideal* (generally unitary) logic gates.  Each has a name starting with <q>G</q>, and is represented as a $d^2\times d^2$ *superoperator* that acts by matrix multiplication on vectors in $\mathcal{B}(\mathcal{H})$.  Matrices are displayed using a heat map that ranges between 1.0 (red) and -1.0 (blue).



In [0]:

ws.GatesTable(gs_target, display_as="boxes")



# System and pyGSTi parameters
This section contains a raw dump of system information and various pyGSTi parameters.  It's purpose is to stamp this report with parameters indicating how exactly GST was run to create it, as well as to record the software environment in within which the report creation was run.  Note that if the core GST computation was done on a machine different from the one that created this report, the software information contained here will be of less value.




### Listing of GST parameters and meta-data.
These parameters and related metadata describe how the GST computation was performed which led to this report.


In [0]:

ws.MetadataTable(gs_final, switchBd.params)



 
### Listing of the software environment.
Note that this describes the software environment of the machine used to generate this report, and not necessarily the machine used to perform the core GST gate set estimation.



In [0]:

ws.SoftwareEnvTable()
